In [1]:
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Subset
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.amp import GradScaler, autocast
import os
import random
import shap

In [2]:
import torch.nn as nn

# Residual block
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x

# ConvMixer model with hard-coded parameters
def ConvMixer():
    dim = 256          # Embedding dimension
    depth = 8          # Number of ConvMixer blocks
    kernel_size = 5    # Kernel size for depthwise convolution
    patch_size = 4     # Patch size for initial convolution
    n_classes = 10     # CIFAR-10 has 10 classes

    return nn.Sequential(
        nn.Conv2d(3, dim, kernel_size=patch_size, stride=patch_size),
        nn.GELU(),
        nn.BatchNorm2d(dim),
        *[nn.Sequential(
                Residual(nn.Sequential(
                    nn.Conv2d(dim, dim, kernel_size, groups=dim, padding="same"),
                    nn.GELU(),
                    nn.BatchNorm2d(dim)
                )),
                nn.Conv2d(dim, dim, kernel_size=1),
                nn.GELU(),
                nn.BatchNorm2d(dim)
        ) for _ in range(depth)],
        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(),
        nn.Linear(dim, n_classes)
    )

In [3]:
import torch
# Load the entire model
model = torch.load('/home/j597s263/scratch/j597s263/Models/Conv_Imagenette.mod', weights_only=False, map_location="cuda:0")

# Move the model to the appropriate device
model = model.to('cuda')

# Set the model to evaluation mode
model.eval()

print("Model loaded successfully!")

Model loaded successfully!


In [4]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to ConvMixer input size
    transforms.ToTensor()
])

# Load the dataset
dataset = datasets.Imagenette(root='/home/j597s263/scratch/j597s263/Datasets/imagenette', download=False, transform=transform)

# Shuffle indices with a fixed random seed for reproducibility
random.seed(42)  # Use any fixed seed for consistency
indices = list(range(len(dataset)))
random.shuffle(indices)

# Split shuffled indices into training and testing
train_indices = indices[:7568]
test_indices = indices[7568:8522]
attack_indices = indices[8522:]

# Create Subsets
train_data = Subset(dataset, train_indices)
test_data = Subset(dataset, test_indices)
attack_data = Subset(dataset, attack_indices)

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)  # Shuffle within batches
test_loader = DataLoader(test_data, batch_size=len(test_data), shuffle=False)  # No shuffle for test set
attack_loader = DataLoader(attack_data, batch_size=5, shuffle=True)

# Print dataset sizes
print(f"Attack samples: {len(attack_data)}")
print(f"Training samples: {len(train_data)}")
print(f"Test samples: {len(test_data)}")

Attack samples: 947
Training samples: 7568
Test samples: 954


In [6]:
'''
import shap
import torch
import numpy as np
import matplotlib.pyplot as plt
# Ensure the model is in evaluation mode
model.eval()

# Select a small batch from the attack dataset to explain
attack_images, _ = next(iter(attack_loader))
attack_images = attack_images[:10]  # Select the first 10 images
attack_images = attack_images.to(device)

# Select background samples from the training data
background_loader = DataLoader(train_data, batch_size=50, shuffle=True)
background_images, _ = next(iter(background_loader))  # Select the first batch
background_images = background_images.to(device)

# Initialize SHAP Gradient Explainer
explainer = shap.GradientExplainer(model, background_images)

# Explain the attack images
shap_values = explainer.shap_values(attack_images)

# Convert attack images to numpy format for visualization
attack_images_np = attack_images.cpu().numpy().transpose(0, 2, 3, 1)  # [N, H, W, C]

'''


'\nimport shap\nimport torch\nimport numpy as np\nimport matplotlib.pyplot as plt\n# Ensure the model is in evaluation mode\nmodel.eval()\n\n# Select a small batch from the attack dataset to explain\nattack_images, _ = next(iter(attack_loader))\nattack_images = attack_images[:10]  # Select the first 10 images\nattack_images = attack_images.to(device)\n\n# Select background samples from the training data\nbackground_loader = DataLoader(train_data, batch_size=50, shuffle=True)\nbackground_images, _ = next(iter(background_loader))  # Select the first batch\nbackground_images = background_images.to(device)\n\n# Initialize SHAP Gradient Explainer\nexplainer = shap.GradientExplainer(model, background_images)\n\n# Explain the attack images\nshap_values = explainer.shap_values(attack_images)\n\n# Convert attack images to numpy format for visualization\nattack_images_np = attack_images.cpu().numpy().transpose(0, 2, 3, 1)  # [N, H, W, C]\n\n'

In [9]:
import shap
import torch
import numpy as np
import matplotlib.pyplot as plt

# Ensure the model is in evaluation mode
model.eval()

# Select a small batch from the attack dataset to explain
attack_images, _ = next(iter(attack_loader))
attack_images = attack_images[:10]  # Select the first 10 images
attack_images = attack_images.to(device)

# Use a subset from the training data as the baseline
baseline_images, _ = next(iter(train_loader))  # Take a batch from train_loader
baseline_images = baseline_images[:1000]  # Limit to 50 images for efficiency
baseline_images = baseline_images.to(device)

# Initialize SHAP Gradient Explainer with the baseline
explainer = shap.GradientExplainer(model, baseline_images)

# Explain the attack images
shap_values = explainer.shap_values(attack_images)

'attack_images = attack_images[:10]  # Select the first 10 images\nattack_images = attack_images.to(device)\n\n# Use a subset from the training data as the baseline\nbaseline_images, _ = next(iter(train_loader))  # Take a batch from train_loader\nbaseline_images = baseline_images[:1000]  # Limit to 50 images for efficiency\nbaseline_images = baseline_images.to(device)\n\n# Initialize SHAP Gradient Explainer with the baseline\nexplainer = shap.GradientExplainer(model, baseline_images)\n\n# Explain the attack images\nshap_values = explainer.shap_values(attack_images)'

In [12]:
model.eval()
attack_images, _ = next(iter(attack_loader))

# Use a subset from the training data as the baseline
baseline_images, _ = next(iter(train_loader))  # Take a batch from train_loader
baseline_images = baseline_images[:1000]  # Limit to 50 images for efficiency
baseline_images = baseline_images.to(device)

explainer = shap.GradientExplainer(model, baseline_images)
shap_values = np.zeros((3, 224, 224, 10))  # (Channels, Height, Width, Classes)

for i in range(947):
    attack_image = attack_images[i]
    attack_image = attack_image.to(device)

    shap_values += explainer.shap_values(attack_image)
    print("1 image processed")

RuntimeError: expand(torch.cuda.FloatTensor{[3, 224, 224]}, size=[224, 224]): the number of sizes provided (2) must be greater or equal to the number of dimensions in the tensor (3)

In [6]:
model.eval()
device='cuda'
# Use a subset from the training data as the baseline
baseline_images, _ = next(iter(train_loader))  # Take a batch from train_loader
baseline_images = baseline_images[:1000]  # Limit to 50 images for efficiency
baseline_images = baseline_images.to(device)

# Initialize SHAP Gradient Explainer
explainer = shap.GradientExplainer(model, baseline_images)

# Initialize aggregation array for SHAP values
shap_values = np.zeros((3, 224, 224, 10))  # (Channels, Height, Width, Classes)

# Process attack images in batches
for batch_images, _ in attack_loader:
    batch_images = batch_images.to(device)

    # Compute SHAP values for the batch
    batch_shap_values = explainer.shap_values(batch_images)  # Shape: (batch_size, 3, 224, 224, 10)

    # Sum the SHAP values into the aggregation array
    for i in range(len(batch_images)):
        shap_values += batch_shap_values[i]

    print(f"{len(batch_images)} images processed in this batch")

# Normalize SHAP values by the number of attack images
shap_values /= len(attack_loader.dataset)

print("SHAP value aggregation completed!")


5 images processed in this batch
5 images processed in this batch
5 images processed in this batch


KeyboardInterrupt: 